# Module Feedback

This system records the responses of students on their learning experience at university.

![rel](https://sqlzoo.net/w/images/7/71/Module-feedback.png)

In [1]:
import $ivy.`org.apache.spark::spark-sql:3.4.0`

import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)

import org.apache.spark._
import org.apache.spark.sql._
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.expressions.Window

val spark = {
    NotebookSparkSession.builder()
    .progress(false)
    .appName("app11")
    // .master("spark://192.168.31.31:7077")
    .master("local[*]")
    .config("spark.sql.warehouse.dir", 
            "hdfs://192.168.31.31:9000/user/hive/warehouse") 
    .config("spark.cores.max", "4") 
    .config("spark.executor.instances", "1") 
    .config("spark.executor.cores", "2") 
    .config("spark.executor.memory", "10g") 
    .config("spark.shuffle.service.enabled", "false") 
    .config("spark.dynamicAllocation.enabled", "false") 
    .config("spark.sql.catalogImplementation", "hive")
    .config("spark.sql.repl.eagerEval.enabled", "true")
    .config("spark.driver.allowMultipleContexts", "true")
    .getOrCreate()
}

Loading spark-stubs, spark-hive
Adding Hive conf dir /opt/hive/conf to classpath
Creating SparkSession


SLF4J: No SLF4J providers were found.
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See https://www.slf4j.org/codes.html#noProviders for further details.


Spark UI

import $ivy.$                                  


import org.apache.log4j.{Level, Logger}

import org.apache.spark._

import org.apache.spark.sql._

import org.apache.spark.sql.types._

import org.apache.spark.sql.functions._

import org.apache.spark.sql.expressions.Window


spark: SparkSession = org.apache.spark.sql.SparkSession@2808f699

In [2]:
import spark.implicits._
def sc = spark.sparkContext
val hiveCxt = new org.apache.spark.sql.hive.HiveContext(sc)

import spark.implicits._

defined function sc
hiveCxt: sql.hive.HiveContext = org.apache.spark.sql.hive.HiveContext@1db6496

In [3]:
// Credit to Aivean
implicit class RichDF(val ds:DataFrame) {
    def showHTML(limit: Int = 50, truncate: Int = 100) = {
        import xml.Utility.escape
        val data = ds.take(limit)
        val header = ds.schema.fieldNames.toSeq        
        val rows: Seq[Seq[String]] = data.map { row =>
          row.toSeq.map {cell =>
            val str = cell match {
              case null => "null"
              case binary: Array[Byte] => binary.map("%02X".format(_)).mkString("[", " ", "]")
              case array: Array[_] => array.mkString("[", ", ", "]")
              case seq: Seq[_] => seq.mkString("[", ", ", "]")
              case _ => cell.toString
            }
            if (truncate > 0 && str.length > truncate) {
              // do not show ellipses for strings shorter than 4 characters.
              if (truncate < 4) str.substring(0, truncate)
              else str.substring(0, truncate - 3) + "..."
            } else {
              str
            }
          }: Seq[String]
        }
    publish.html(s""" <table>
                <tr>
                 ${header.map(h => s"<th>${escape(h)}</th>").mkString}
                </tr>
                ${rows.map {row =>
                  s"<tr>${row.map{c => s"<td>${escape(c)}</td>" }.mkString}</tr>"
                }.mkString}
            </table>
        """)
    }
}

defined class RichDF

In [4]:
val insspr = hiveCxt.table("sqlzoo.INS_SPR")
val insmod = hiveCxt.table("sqlzoo.INS_MOD")
val insprs = hiveCxt.table("sqlzoo.INS_PRS")
val camsmo = hiveCxt.table("sqlzoo.CAM_SMO")
val insres = hiveCxt.table("sqlzoo.INS_RES")
val insque = hiveCxt.table("sqlzoo.INS_QUE")
val inscat = hiveCxt.table("sqlzoo.INS_CAT")


insspr: DataFrame = [spr_code: string, spr_fnm1: string ... 1 more field]
insmod: DataFrame = [mod_code: string, mod_name: string ... 1 more field]
insprs: DataFrame = [prs_code: string, prs_fnm1: string ... 1 more field]
camsmo: DataFrame = [spr_code: string, mod_code: string ... 2 more fields]
insres: DataFrame = [spr_code: string, mod_code: string ... 4 more fields]
insque: DataFrame = [que_code: string, cat_code: string ... 2 more fields]
inscat: DataFrame = [cat_code: string, cat_name: string ... 1 more field]

## 1. Find the student name from a matriculation number

**Find the name of the student with number 50200100**

In [5]:
(insspr.filter(col("SPR_CODE")==="50200100")
 .select("SPR_FNM1", "SPR_SURN").showHTML())

SPR_FNM1,SPR_SURN
Tom,Cotton


## 2. Find the modules studied by a student

**Show the module code and module name for modules studied by the student with number 50200100 in session 2016/7 TR1**

In [6]:
(camsmo.filter((col("SPR_CODE")==="50200100") && 
               (trim(col("AYR_CODE"))==="2016/7") && 
               (col("PSL_CODE")==="TR1"))
 .join(insmod, "MOD_CODE")
 .select("MOD_CODE", "MOD_NAME")
 .showHTML())

MOD_CODE,MOD_NAME
CSN08101,Systems and Services
INF08104,Database Systems
SET08108,Software Development 2


## 3. Find the modules and module leader studied by a student

**Show the module code and module name and details of the module leader for modules studied by the student with number 50200100 in session 2016/7 TR1**

In [7]:
(camsmo.filter((col("SPR_CODE")==="50200100") && 
               (trim(col("AYR_CODE"))==="2016/7") && 
               (col("PSL_CODE")==="TR1"))
 .join(insmod, "MOD_CODE")
 .join(insprs, "PRS_CODE")
 .select("MOD_CODE", "MOD_NAME", "PRS_CODE", "PRS_FNM1", "PRS_SURN")
 .showHTML())

MOD_CODE,MOD_NAME,PRS_CODE,PRS_FNM1,PRS_SURN
CSN08101,Systems and Services,40000008,James,Jackson
INF08104,Database Systems,40000036,Andrew,Cumming
SET08108,Software Development 2,40000408,Neil,Urquhart


## 4. Show the scores for module SET08108

**Show the Percentage of students who gave 4 or 5 to module SET08108 in session 2016/7 TR1**

(note that this is not real data, these responses were randomly generated)

In [8]:
(insres.filter((col("MOD_CODE")==="SET08108") && 
               (trim(col("AYR_CODE"))==="2016/7") &&
               (col("PSL_CODE")==="TR1"))
 .withColumn("score", floor(col("RES_VALU")/4))
 .groupBy("QUE_CODE")
 .agg(sum("score").alias("score"), 
      count("score").alias("n"))
 .withColumn("pct", round(lit(100) * col("score") / col("n"), 0))
 .join(insque, "QUE_CODE")
 .join(inscat, "CAT_CODE")
 .select("QUE_CODE", "QUE_TEXT", "CAT_NAME", "pct")
 .orderBy("QUE_CODE")
 .showHTML())

QUE_CODE,QUE_TEXT,CAT_NAME,pct
1.1,Staff are good at explaining things.,Learning and Teaching,89.0
1.2,Staff made the subject interesting.,Learning and Teaching,82.0
1.3,The module was intellectually stimulating.,Learning and Teaching,82.0
1.4,The aims and objectives were clearly stated.,Learning and Teaching,89.0
1.5,The module was well-organised and ran smoothly.,Learning and Teaching,78.0
1.6,The pace was appropriate.,Learning and Teaching,80.0
1.7,The level was appropriate.,Learning and Teaching,82.0
1.8,The workload was managable.,Learning and Teaching,78.0
1.9,I was able to contact module staff when I needed to.,Learning and Teaching,76.0
2.1,The assessment requirements were clearly stated.,Assessment and Feedback,84.0


## 5. Show the frequency chart for module SET08108 for question 4.1

**For each response 1-5 show the number of students who gave that response (Module SET08108, 2016/7, TR1)**

(note that this is not real data, these responses were randomly generated)

In [9]:
(insres.filter((col("MOD_CODE")==="SET08108") && 
               (trim(col("AYR_CODE"))==="2016/7") &&
               (col("PSL_CODE")==="TR1") &&
               (col("QUE_CODE")==="4.1"))
 .groupBy("MOD_CODE", "RES_VALU")
 .count()
 .showHTML())

MOD_CODE,RES_VALU,count
SET08108,2,6
SET08108,5,39
SET08108,4,10


In [10]:
spark.stop()